In [7]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Conv2D,MaxPooling2D, Flatten

from keras.utils import multi_gpu_model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import matplotlib
import pylab as plt
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
import time
from keras.utils import np_utils
from sklearn import preprocessing as pp

import h5py

X_train = np.load("./data/train_features.pkl")
Y_train = np.load("./data/train_labels.pkl")
X_test = np.load("./data/test_features.pkl")
Y_test = np.load("./data/test_labels.pkl")



cnn2d_shallow = Sequential([Conv2D(64, (3,3), padding="same",input_shape=(8,8,3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid'),
])

cnn2d_moderate = Sequential([Conv2D(64, (3,3), padding="same",input_shape=(8,8,3)),
    Activation('relu'),
    Conv2D(128, (3,3), padding="same"),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid'),
])

cnn2d_deep = Sequential([Conv2D(64, (3,3), padding="same",input_shape=(8,8,3)),
    Activation('relu'),
    Conv2D(128, (3,3), padding="same"),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(256, (3,3), padding="same"),
    Activation('relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dense(64),
    Activation('relu'),
    Dense(32),
    Activation('relu'),
    Dropout(0.5),
    Dense(1),
    Activation('sigmoid'),
])
print(cnn2d_moderate.summary())
print(cnn2d_deep.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 8, 8, 64)          1792      
_________________________________________________________________
activation_19 (Activation)   (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
activation_20 (Activation)   (None, 8, 8, 128)         0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_12 (Dense)             (None, 128)               262272    
__________

In [8]:
cnn2d_shallow.compile(loss='binary_crossentropy', optimizer="adam",metrics=['accuracy'])
cnn2d_moderate.compile(loss='binary_crossentropy', optimizer="adam",metrics=['accuracy'])
cnn2d_deep.compile(loss='binary_crossentropy', optimizer="adam",metrics=['accuracy'])

# train
cnn2d_shallow.fit(X_train, Y_train, batch_size=64,epochs=10)
print("------------------------------------------------------------------------------------------------------")
cnn2d_moderate.fit(X_train, Y_train, batch_size=64,epochs=10)
print("------------------------------------------------------------------------------------------------------")
cnn2d_deep.fit(X_train, Y_train, batch_size=64,epochs=10)
print("------------------------------------------------------------------------------------------------------")

# serialize weights to HDF5
cnn2d_shallow.save("./models/NSL_KDD_Train_shallow.h5")
cnn2d_moderate.save("./models/NSL_KDD_Train_moderate.h5")
cnn2d_deep.save("./models/NSL_KDD_Train_deep.h5")

Epoch 1/10
125972/125972 [==============================] - 23s 180us/step - loss: 3.0040 - acc: 0.5289
Epoch 2/10
125972/125972 [==============================] - 22s 175us/step - loss: 0.6909 - acc: 0.5346
Epoch 3/10
125972/125972 [==============================] - 20s 161us/step - loss: 0.6909 - acc: 0.5346
Epoch 4/10
125972/125972 [==============================] - 20s 159us/step - loss: 0.6908 - acc: 0.5346
Epoch 5/10
125972/125972 [==============================] - 20s 160us/step - loss: 0.6908 - acc: 0.5346
Epoch 6/10
125972/125972 [==============================] - 20s 159us/step - loss: 0.6908 - acc: 0.5346
Epoch 7/10
125972/125972 [==============================] - 20s 160us/step - loss: 0.6908 - acc: 0.5346
Epoch 8/10
125972/125972 [==============================] - 20s 160us/step - loss: 0.6908 - acc: 0.5346
Epoch 9/10
125972/125972 [==============================] - 20s 160us/step - loss: 0.6908 - acc: 0.5346
Epoch 10/10
125972/125972 [==============================] - 20s

In [5]:
shallow_score = cnn2d_shallow.evaluate(X_test, Y_test, verbose=0)
moderate_score = cnn2d_shallow.evaluate(X_test, Y_test, verbose=0)
deep_score = cnn2d_shallow.evaluate(X_test, Y_test, verbose=0)

#print loss and accuracy
print('Shallow Test loss:', shallow_score[0])
print('Shallow Test accuracy:', shallow_score[1])

print('Moderate Test loss:', moderate_score[0])
print('Moderate Test accuracy:', moderate_score[1])

print('Deep Test loss:', deep_score[0])
print('Deep Test accuracy:', deep_score[1])

ValueError: Error when checking target: expected activation_3 to have shape (1,) but got array with shape (38,)

### Evaluation Metrics - F1, Confusion Matrix - (Normal as 0 and Anomaly as 1)

In [12]:
from sklearn.metrics import f1_score
import pandas as pd 
from sklearn.metrics import classification_report


shallow_pred = cnn2d_shallow.predict_classes(X_test)
moderate_pred = cnn2d_moderate.predict_classes(X_test)
deep_pred = cnn2d_deep.predict_classes(X_test)

print(classification_report(Y_test, shallow_pred))
print(classification_report(Y_test, moderate_pred))
print(classification_report(Y_test, deep_pred))

print(f1_score(Y_test, shallow_pred))
print(f1_score(Y_test, moderate_pred))
print(f1_score(Y_test, deep_pred))

print(pd.crosstab(shallow_pred,))
print(pd.crosstab(moderate_pred))
print(pd.crosstab(deep_pred))

             precision    recall  f1-score   support

          0       0.43      1.00      0.60      9711
          1       0.00      0.00      0.00     12832

avg / total       0.19      0.43      0.26     22543

             precision    recall  f1-score   support

          0       0.43      1.00      0.60      9711
          1       0.00      0.00      0.00     12832

avg / total       0.19      0.43      0.26     22543

             precision    recall  f1-score   support

          0       0.43      1.00      0.60      9711
          1       0.00      0.00      0.00     12832

avg / total       0.19      0.43      0.26     22543

0.0
0.0
0.0


C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda\envs\tensorflow-cpu\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


TypeError: crosstab() missing 1 required positional argument: 'columns'